# Demo of WFST-based POS Tagging
## for Intern3 Students
by Ye Kyaw Thu, Lab Leader, LST Lab., Myanmar  
Date: 12 July 2025

## Shell Scripts

ဒီ Lab မှာက OpenFST command တွေကို သုံးပြီး NLP field မှာ အရေးကြီးတဲ့ task တစ်ခုဖြစ်တဲ့ POS (Part-of-Speech) Tagging ကို လက်တွေ့ လုပ်ပြသွားမယ်။  
ကိုယ့်စက်ထဲမှာ လိုက်လုပ်ဖို့အတွက်က လိုအပ်တဲ့ OpenFST နဲ့ တခြား python library တွေက ကြိုတင် installation လုပ်ထားရလိမ့်မယ်။  

In [3]:
!pwd

/home/ye/exp/wfst_pos


Bash shell script တွေက အောက်ပါအတိုင်း

In [28]:
!ls ./shell_script/*.sh

./shell_script/build_fst.sh	   ./shell_script/rebuild.sh
./shell_script/find_blank_line.sh  ./shell_script/run_tagger.sh
./shell_script/pipe2space.sh	   ./shell_script/run_wfst_pos.sh
./shell_script/preprocess.sh	   ./shell_script/test_tagger.sh


Python နဲ့ perl script တွေက အောက်ပါအတိုင်း

In [108]:
!ls ./script/*.{py,pl}

./script/build_lexicon.py  ./script/pos_evaluation.py
./script/build_pos_lm.py   ./script/prepare_test_data.py
./script/col2line.pl	   ./script/preprocess.py
./script/evaluate.py


## Dataset 

POS Tagging အတွက် ဒေတာက LU Lab. ရဲ့ ဒေတာဖြစ်တဲ့ myPOS (Version 3.0) ကိုသုံးပါမယ်။  
ဒေတာ download လုပ်တဲ့ အဆင့်ကို ကျော်ပါမယ်။  
ဒေတာက publicly available ပါ။ အောက်ပါ လင့်ကနေ ယူသုံးနိုင်ပါတယ်။  
[https://github.com/ye-kyaw-thu/myPOS/tree/master/corpus-ver-3.0](https://github.com/ye-kyaw-thu/myPOS/tree/master/corpus-ver-3.0)  

In [10]:
!ls ./data/

corpus.txt  mypos-ver.3.0.shuf.txt  otest.1k.txt  otest.txt


In [13]:
!wc ./data/*

  43196  537233 9581544 ./data/mypos-ver.3.0.shuf.txt
   1000   12825  229758 ./data/otest.1k.txt
  44196  550058 9811302 total


## Data Cleaning

myPOS ဒေတာမှာက compound word တွေကို | (pipe) သင်္ကေတနဲ့တွဲထားတာမို့လို့ အဲဒီ pipe တွေကို ဖြုတ်ခဲ့တယ်။  

In [16]:
!grep "|" ./data/mypos-ver.3.0.shuf.txt | head

လူ/n တိုင်း/part တွင်/ppm သင့်မြတ်/v လျော်ကန်/v စွာ/part ကန့်သတ်/v ထား/part သည့်/part အလုပ်/n လုပ်/v ချိန်/n အပြင်/conj ၊/punc လစာ/n နှင့်တကွ/conj အခါ/n ကာလ/n အားလျော်စွာ/ppm သတ်မှတ်/v ထား/part သည့်/part အလုပ်/n|အားလပ်ရက်/n များ/part ပါဝင်/v သည့်/part အနားယူခွင့်/n နှင့်/conj အားလပ်ခွင့်/n ခံစားပိုင်ခွင့်/n ရှိ/v သည်/ppm ။/punc
စာပြန်ပွဲ/n ဆို/v တာ/part က/ppm အာဂုံဆောင်/v အလွတ်ကျက်/v ထား/part တဲ့/part ပိဋကတ်သုံးပုံ/n|စာပေ/n တွေ/part ကို/ppm စာစစ်/v|သံဃာတော်ကြီး/n တွေ/part ရဲ့/ppm ရှေ့/n မှာ/ppm အလွတ်/adv ပြန်/v ပြီး/part ရွတ်ပြ/v ရ/part တာ/part ပေါ့/part ။/punc
၂ဝ/num ရာစု/n မြန်မာ့/n|သမိုင်း/n သန်းဝင်းလှိုင်/n ၊/punc ၂ဝဝ၉/num ခု/part ၊/punc မေ/n|လ/n ၊/punc ကံကော်ဝတ်ရည်/n|စာပေ/n ။/punc
၅/num ရက်/n လောက်/part ပဲ/part အချိန်/n ရ/v တယ်/ppm ။/punc ရန်ကုန်/n မှာ/ppm ဘာ/pron လည်/v|စရာ/part တွေ/part ရှိ/v သလဲ/part ပြော/v ပါ/part ဦး/part ။/punc
မြို့/n|လယ်/n တွင်/ppm ကား/part ကျိုက်သုတ်/n|စေတီ/n ရှိ/v သည်/ppm ။/punc
ကျော်သူ/n ၁၉၅၉/num နိုဝင်ဘာ/n|လ/n ၂/num ရက်/n တွင်/ppm မွေးဖွား/v သည်/ppm ။/p

**Linux/Unix မှာ powerful text processing command တစ်ခုဖြစ်တဲ့ sed (string editor) ကိုသုံးပြီး pipe တွေကို ဖြုတ်မယ်။**  

In [15]:
!sed 's/|/ /g' ./data/mypos-ver.3.0.shuf.txt > ./data/corpus.txt

grep command နဲ့ pipe character ပါနေတဲ့ စာကြောင်းတွေ ရှိနေသေးသလား ဆိုတာကို confirm လုပ်မယ်။  

In [20]:
!grep "|" ./data/corpus.txt | head

In [21]:
!wc ./data/corpus.txt

  43196  564517 9581544 ./data/corpus.txt


open-test ဒေတာကိုဖိုင်ရဲ့ pipe တွေကိုလည်း ရှင်းခဲ့တယ်။  

In [26]:
!grep "|" ./data/otest.1k.txt | head

အေးချမ်း/v|မှု/part နဲ့/conj စည်းကမ်း/n ကို/ppm တည်မြဲ/v အောင်/part ထိန်းသိမ်း/v သည်/ppm ။/punc
ကျွန်တော်/pron ငွေ/n|အရွက်/n|ကြီး/adj တွေ/part ရ/v မလား/part ။/punc
ဒီ/adj နေ့/n ကမ္ဘာ့/n|စံချိန်သစ်/n တင်/v ခဲ့/part တယ်/ppm ။/punc
အာဖရိက/n|တိုက်/n ၊/punc ဆီးရီးယား/n နှင့်/conj ပါးရှား/n|နိုင်ငံ/n ရှိ/v သဲကန္တာရ/n များ/part နှင့်/conj အိန္ဒိယ/n|နိုင်ငံ/n အလယ်/n|ပိုင်း/part ရှိ/v လွင်ပြင်/n များ/part တွင်/ppm မြင်းကျား/n နှင့်/conj မြည်း/n ရိုင်း/adj များ/part ကို/ppm တွေ့/v ရှိ/part နိုင်/part သည်/ppm ။/punc
အင်္ဂါ/n|ဂြိုဟ်/n ပေါ်/n တွင်/ppm ယခု/n လက်ရှိ/adj အချိန်/n ၌/ppm ဝင်ရိုးစွန်း/n|ဒေသ/n များ/part မှ/part အပ/n ရေ/n မ/part ရှိ/v ပေ/part ။/punc
ဗမာ/n|လူမျိုး/n တို့/part သည်/ppm တိဘက်/n မြန်မာ/n Tbeto/fw Burman/fw လူမျိုးနွယ်စု/n|ကြီး/part မှ/ppm အကြီးဆုံး/adj သော/part လူမျိုး/n ဖြစ်/v သည်/ppm ။/punc
ရန်ကုန်/n|ဆေး/n|တက္ကသိုလ်/n ကို/ppm ဘယ်/adj မှတ်တိုင်/n မှာ/ppm ဆင်း/v ရ/part မှာ/ppm လဲ/part ။/punc
အချို့/adj ငါး/n များ/part သည်/ppm မိမိ/pron တို့/part နေထိုင်/v|ရာ/part ပတ်ဝန်းကျင်/n 

In [27]:
!grep "|" ./data/otest.1k.txt | wc

    257    5051  103365


In [22]:
!sed 's/|/ /g' ./data/otest.1k.txt > ./data/otest.txt

In [23]:
!wc ./data/otest.txt

  1000  13468 229758 ./data/otest.txt


In [24]:
!grep "|" ./data/otest.txt

## Hint
လက်တွေ့ experiment တွေ လုပ်ကြတဲ့အခါမှာ ဒေတာပြောင်းလုပ်ရတာမျိုးလည်း ရှိလို့ ပြီးတော့ အသုံးဝင်တဲ့ command တွေက အလွတ်မှတ်မထားနိုင်ရင် bash shell script နဲ့ ရေးထားတာမျိုး လုပ်လို့ ရတယ်။  

In [30]:
!cat ./shell_script/pipe2space.sh

#!/bin/bash

sed 's/|/ /g' ../data/mypos-ver.3.0.shuf.txt > ../data/corpus.txt
sed 's/|/ /g' ../data/otest.1k.txt > ../data/otest.txt


ကိုယ်သုံးမယ့် ဒေတာပေါ်မူတည်ပြီး corpus ထဲမှာ blank line တွေပါတတ်တာမျိုးလည်း ရှိတာမို့ အဲဒါတွေကို ကြိုတင်ဖယ်ရှားထားတာက ကောင်းတယ်။ မဟုတ်ရင် experiment အတွက် သုံးခဲ့တဲ့ corpus စာကြောင်းအရေအတွက်ကို တင်ပြတဲ့အခါမှာ blank line တွေကိုပါ ထည့်ရေတွက်ထားတာမျိုးလည်း ဖြစ်သွားနိုင်လို့။ သုံးမယ့် code တွေအပေါ်မူတည်ပြီး လက်တွေ့ run ရတဲ့အခါမှာ blank line တွေကြောင့် error ဖြစ်တာမျိုးလည်း ရှိတယ်။  

In [31]:
!cat ./shell_script/find_blank_line.sh

#!/bin/bash

# for printout filename and lineno of blank lines 
# written by Ye, NECTEC, Thailand
# How to run: ./find-blank-lines.sh <filename or regular expression>
# e.g. ./find-blank-lines.sh train.my
# e.g. ./find-blank-lines.sh 't*.*' 
# Note: when you want to pass special characters such as *, you should use single quote!!!

#find . -name "$1"  | xargs grep -E --line-number --with-filename '^$';
find . -wholename "$1"  | xargs grep -E --line-number --with-filename '^$';


## Preprocessing
**တကယ်တမ်း POS Tagging အတွက် သုံးမှာက corpus.txt နဲ့ otest.txt နှစ်ဖိုင်ပါ။**  
အဲဒီဖိုင်နှစ်ဖိုင်ကနေပဲ WFST အတွက် လိုအပ်တဲ့ format အဖြစ်ပြောင်းဖို့အတွက် preprocessing လုပ်သွားတာမျိုးပါ။  

In [32]:
!cat ./shell_script/preprocess.sh

#!/bin/bash

mkdir ../fst
# preprocess
python ../script/preprocess.py ../data/corpus.txt ../fst/pairs.txt ../fst/words.syms ../fst/tags.syms



In [33]:
!pwd

/home/ye/exp/wfst_pos


In [35]:
cd ./shell_script/

/home/ye/exp/wfst_pos/shell_script


In [36]:
!./preprocess.sh

Preprocessing complete. Output files: ../fst/pairs.txt, ../fst/words.syms, ../fst/tags.syms


ထွက်လာတဲ့ output ဖိုင်တွေကို စစ်ဆေးကြည့်မယ်။  

In [37]:
!wc ../fst/*

  564517  1129034  9581272 ../fst/pairs.txt
      22       44      197 ../fst/tags.syms
   24827    49654   756965 ../fst/words.syms
  589366  1178732 10338434 total


In [38]:
!head ../fst/pairs.txt

၁၉၆၂	num
ခုနှစ်	n
ခန့်မှန်း	v
သန်းခေါင်စာရင်း	n
အရ	ppm
လူဦးရေ	n
၁၁၅၉၃၁	num
ယောက်	part
ရှိ	v
သည်	ppm


In [39]:
!head ../fst/tags.syms

<eps>	0
<unk>	1
abb	2
adj	3
adv	4
conj	5
fw	6
int	7
n	8
n/pron	9


In [40]:
!head ../fst/words.syms

<eps>	0
!	1
"	2
%	3
'	4
''	5
(	6
)	7
*	8
+	9


## Build FST

In [51]:
!cat ./build_fst.sh

#!/bin/bash

# Default to bigram if not specified
NGRAM=${1:-2}

# Step 1: Build lexicon FST
python ../script/build_lexicon.py ../fst/pairs.txt ../fst/lexicon.fst.txt ../fst/words.syms ../fst/tags.syms
fstcompile --isymbols=../fst/words.syms --osymbols=../fst/tags.syms ../fst/lexicon.fst.txt ../fst/lexicon.fst

# Step 2: Build POS LM FST with specified ngram
python ../script/build_pos_lm.py ../fst/tags.syms ../fst/pairs.txt ../fst/pos_lm.fst.txt $NGRAM
fstcompile --isymbols=../fst/tags.syms --osymbols=../fst/tags.syms ../fst/pos_lm.fst.txt ../fst/pos_lm.fst

# Step 3: Compose lexicon and LM
fstarcsort --sort_type=olabel ../fst/lexicon.fst > ../fst/lexicon_sorted.fst
fstarcsort --sort_type=ilabel ../fst/pos_lm.fst > ../fst/pos_lm_sorted.fst
fstcompose ../fst/lexicon_sorted.fst ../fst/pos_lm_sorted.fst ../fst/pos_tagger.fst

echo "FST model built: pos_tagger.fst (using ${NGRAM}-gram POS LM)"



In [42]:
!./build_fst.sh

FST model built: pos_tagger.fst (using 2-gram POS LM)


output အနေနဲ့ထွက်လာတဲ့ FST ဖိုင်တွေကို လေ့လာကြည့်ရအောင် ...  

In [45]:
!ls ../fst/

lexicon.fst	    pairs.txt	    pos_lm_sorted.fst  words.syms
lexicon.fst.txt     pos_lm.fst	    pos_tagger.fst
lexicon_sorted.fst  pos_lm.fst.txt  tags.syms


In [46]:
!head -n 30 ../fst/lexicon.fst.txt

0 0 ၁၉၆၂ num -1.0
0 0 ခုနှစ် n -0.9937238493723849
0 0 ခုနှစ် tn -0.006276150627615063
0 0 ခန့်မှန်း v -0.9791666666666666
0 0 ခန့်မှန်း n -0.020833333333333332
0 0 သန်းခေါင်စာရင်း n -1.0
0 0 အရ ppm -0.9932432432432432
0 0 အရ part -0.0033783783783783786
0 0 အရ conj -0.0033783783783783786
0 0 လူဦးရေ n -1.0
0 0 ၁၁၅၉၃၁ num -1.0
0 0 ယောက် part -0.9868035190615836
0 0 ယောက် n -0.007331378299120235
0 0 ယောက် v -0.002932551319648094
0 0 ယောက် tn -0.001466275659824047
0 0 ယောက် ppm -0.001466275659824047
0 0 ရှိ v -0.9368649008828919
0 0 ရှိ part -0.06296851574212893
0 0 ရှိ n -0.0001665833749791771
0 0 သည် ppm -0.9972071256038647
0 0 သည် part -0.002566425120772947
0 0 သည် adj -7.548309178743961e-05
0 0 သည် pron -7.548309178743961e-05
0 0 သည် n -7.548309178743961e-05
0 0 ။ punc -0.9998238816484678
0 0 ။ part -8.805917576611483e-05
0 0 ။ n -2.2014793941528707e-05
0 0 ။ sb -4.4029587883057415e-05
0 0 ။ ppm -2.2014793941528707e-05
0 0 လူ n -0.9775510204081632


In [47]:
!head -n 30 ../fst/pos_lm.fst.txt 

0 1 num num 0
1 1 n n -0.6479299899023898
1 1 part part -0.13917872770111073
1 1 v v -0.035005048805116123
1 1 adj adj -0.026758667115449344
1 1 sb sb -0.017502524402558062
1 1 ppm ppm -0.07303938067990576
1 1 conj conj -0.011612251767081791
1 1 tn tn -0.0013463480309660047
1 1 num num -0.01666105688320431
1 1 punc punc -0.024739145069000337
1 1 fw fw -0.002019522046449007
1 1 adv adv -0.002019522046449007
1 1 pron pron -0.002019522046449007
1 1 abb abb -0.0001682935038707506
0 1 n n 0
1 1 v v -0.14758727317112866
1 1 ppm ppm -0.2667507527056717
1 1 num num -0.021995280332004232
1 1 part part -0.16896411424851493
1 1 conj conj -0.02246724713158109
1 1 n n -0.25762877370005693
1 1 punc punc -0.03546260883717146
1 1 tn tn -0.024184229799007244
1 1 adv adv -0.015542354951582717
1 1 adj adj -0.02674749776222638
1 1 pron pron -0.007421271055415412
1 1 fw fw -0.0045162340304337216
1 1 abb abb -0.00048824151680364555
1 1 sb sb -0.00018715924810806412


In [49]:
!fstprint --isymbols=../fst/words.syms --osymbols=../fst/tags.syms ../fst/pos_tagger.fst | head -n 30

0	1	A	abb	-0.100000001
0	1	CAAC	abb	-1
0	1	D	abb	-0.75
0	1	G	abb	-0.125
0	1	L	abb	-0.166666672
0	1	N	abb	-0.5
0	1	QQ	abb	-1
0	1	U	abb	-0.166666672
0	1	W	abb	-0.333333343
0	1	YUFL	abb	-1
0	1	ဂျီ	abb	-0.125
0	1	ဂျီစီဘီအေ	abb	-1
0	1	ဂျီဒီ	abb	-1
0	1	ဂျီဒီပီ	abb	-1
0	1	ဂျီပီအိုင်	abb	-1
0	1	ဂျီအင်ပီ	abb	-1
0	1	ဂျီအိုအက်စ်	abb	-1
0	1	ဂျေစီဘီ	abb	-1
0	1	ဂျေအေအယ်လ်	abb	-1
0	1	ငြိမ်ပိ	abb	-1
0	1	စကန	abb	-1
0	1	စီ	abb	-0.00884955749
0	1	စီစီတီဗွီ	abb	-1
0	1	စီဒီ	abb	-1
0	1	စီဒီအမ်အေ	abb	-1
0	1	စီအင်အင်	abb	-1
0	1	စီအိတ်စ်	abb	-1
0	1	စီအီးအို	abb	-1
0	1	တစည	abb	-1
0	1	တီ	abb	-0.25


## Learn Well From Python Codes

တကယ်တမ်း အခုလိုမျိုး FST ဖိုင် format အဖြစ်ပြောင်းထုတ်ဖို့က ကိုယ်တိုင် coding လုပ်ခဲ့ကြရတယ်။ ခုချိန်မှာက LLM တွေလည်း ရှိနေပြီမို့ အများကြီး လုပ်ရကိုင်ရ လွယ်ကူလာပါပြီ။  
### build_lexicon.py

In [52]:
!cat ../script/build_lexicon.py

"""
for building FST lexicon
Written by Ye Kyaw Thu, LU Lab., Myanmar.
Last update: 10 July 2025
Usage:  
    python ./build_lexicon.py ../fst/pairs.txt ../fst/lexicon.fst.txt ../fst/words.syms ../fst/tags.syms
"""

import sys
from collections import defaultdict

def build_lexicon(pairs_file, lexicon_fst_file, words_syms, tags_syms):
    """Create a weighted lexicon FST (word -> POS) in OpenFST text format."""
    counts = defaultdict(lambda: defaultdict(int))
    
    # Read word-POS pairs and count (word, tag) frequencies
    with open(pairs_file, 'r', encoding='utf-8') as f:
        for line in f:
            word, tag = line.strip().split('\t')
            counts[word][tag] += 1

    # Write lexicon FST (weights = negative log probabilities)
    with open(lexicon_fst_file, 'w', encoding='utf-8') as f:
        for word, tags in counts.items():
            total = sum(tags.values())
            for tag, cnt in tags.items():
                weight = -1 * (cnt / total)  # Tropical semi

### build_pos_lm.py

In [54]:
!cat ../script/build_pos_lm.py

"""
for building FST LM
Written by Ye Kyaw Thu, LU Lab., Myanmar.
Last update: 10 July 2025
Usage:
    python ./build_pos_lm.py ../fst/tags.syms ../fst/pairs.txt ../fst/pos_lm.fst.txt 2
"""

import sys
from collections import defaultdict

def build_pos_lm(tags_syms, pairs_file, output_file, ngram=2):
    """Create POS ngram model (2-gram or 3-gram) from training data."""
    pos_counts = defaultdict(lambda: defaultdict(int))
    total = defaultdict(int)
    
    # Read POS sequences from pairs.txt
    with open(pairs_file, 'r', encoding='utf-8') as f:
        history = []  # For ngram history
        for line in f:
            _, tag = line.strip().split('\t')
            
            if ngram == 2:
                # Bigram model
                if len(history) == 1:
                    prev_tag = history[0]
                    pos_counts[prev_tag][tag] += 1
                    total[prev_tag] += 1
                history = [tag]
                
            elif ngram == 3:
          

## POS Tagging  

In [55]:
!cat ./run_tagger.sh

#!/bin/bash

# Written by Ye Kyaw Thu, LU Lab., Myanmar
# Last updated: 10 July 2025
# How to run: time ./run_tagger.sh otest.txt words.syms tags.syms pos_tagger.fst > otest.hyp  

rm ../fst/tagged_output.txt; 

TEST_FILE="$1"
WORDS_SYMS="$2"
TAGS_SYMS="$3"
POS_TAGGER_FST="$4"

# Create temporary directory
TMPDIR=$(mktemp -d)
trap 'rm -rf "$TMPDIR"' EXIT

# Sort the POS tagger FST once
fstarcsort --sort_type=ilabel "$POS_TAGGER_FST" > "$TMPDIR/pos_tagger_sorted.fst"

# Process each sentence individually
sentence_num=0
while read -r sentence; do
    if [ -z "$sentence" ]; then
        continue  # Skip empty lines
    fi
    
    # Store original sentence
    echo "$sentence" > "$TMPDIR/current_sentence.txt"
    
    # Create FST for current sentence
    python ../script/prepare_test_data.py "$TMPDIR/current_sentence.txt" "$TMPDIR/sentence.fst.txt" "$WORDS_SYMS"
    
    # Compile and sort sentence FST
    fstcompile --isymbols="$WORDS_SYMS" --osymbols="$WORDS_SYMS" \
        "$TMPDIR/se

### Running run_tagger.sh

run_tagger ကို run တဲ့အခါမှာ ကိုယ့်ရဲ့ test data ဖိုင်ပမာဏအပေါ်ကို မူတည်ပြီး စောင့်ရလိမ့်မယ်။  
path တွေကို ဂရုစိုက် စစ်ပါ။  

In [58]:
!time ./run_tagger.sh ../data/otest.txt ../fst/words.syms ../fst/tags.syms ../fst/pos_tagger.fst > ../fst/otest.hyp 


real	1m26.144s
user	1m17.298s
sys	0m19.746s


In [60]:
!ls ../fst/

lexicon.fst	    otest.hyp	pos_lm.fst.txt	   tagged_output.txt
lexicon.fst.txt     pairs.txt	pos_lm_sorted.fst  tags.syms
lexicon_sorted.fst  pos_lm.fst	pos_tagger.fst	   words.syms


ဆောက်လိုက်တဲ့ fst ဖိုင်ရဲ့ size ကိုလည်း စစ်ဆေးပါ။ ဥပမာ filesize က zero ဆိုရင်တော့ တနေရာရာမှာ လွဲနေပြီ။  
OpenFST ကို သုံးရတဲ့အခါမှာ error ကို trace လိုက်တာက ခက်တယ်။ အလွယ်ဆုံးက ထွက်လာတဲ့ fst ဖိုင်တွေရဲ့ size က zero ဖြစ်နေလား ဆိုတာကို စစ်ကြည့်တာက တနည်းပါပဲ။  

In [61]:
!wc ../fst/*

    1557     3942   441374 ../fst/lexicon.fst
   27582   137907  1041303 ../fst/lexicon.fst.txt
    1557     3072   441374 ../fst/lexicon_sorted.fst
   14468    26936   229592 ../fst/otest.hyp
  564517  1129034  9581272 ../fst/pairs.txt
      32       57     3914 ../fst/pos_lm.fst
     240     1197     7688 ../fst/pos_lm.fst.txt
      32       55     3914 ../fst/pos_lm_sorted.fst
   27639    72262  7686954 ../fst/pos_tagger.fst
   14468    26936   229592 ../fst/tagged_output.txt
      22       44      197 ../fst/tags.syms
   24827    49654   756965 ../fst/words.syms
  676941  1451096 20424139 total


**နောက်ထပ် စစ်ဆေးလို့ ရတာက fstinfo commmand ကိုသုံးပြီး arc အရေအတွက်ကို ကြည့်တာမျိုးပါ။**  

In [62]:
!fstinfo ../fst/lexicon.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       1
# of arcs                                         27581
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          1.27606
output label multiplicity                         9167.74
# of accessible states                            1
# of coaccessible states                          1
# of connected states                             1
# of connected components                         1
# of strongly conn components                     1
input matcher                 

**Language model ကိုလည်း fstinfo command နဲ့ လေ့လာကြည့်ရအောင်။**  

In [63]:
!fstinfo ../fst/pos_lm.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       2
# of arcs                                         239
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          13.2343
output label multiplicity                         13.2343
# of accessible states                            2
# of coaccessible states                          2
# of connected states                             2
# of connected components                         1
# of strongly conn components                     2
input matcher                   

**POS Tagger FST model ဖိုင်ကိုလည်း fstinfo command နဲ့ လေ့လာကြည့်ရအောင်။**  

In [64]:
!fstinfo ../fst/pos_tagger.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       2
# of arcs                                         480429
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          19.7019
output label multiplicity                         151254
# of accessible states                            2
# of coaccessible states                          2
# of connected states                             2
# of connected components                         1
# of strongly conn components                     2
input matcher                 

**ကိုယ့်ရဲ့ WFST POS Tagger က တကယ်တမ်း အလုပ် လုပ်တယ် မလုပ်ဘူး ဆိုတာကို အမြန်ဆုံး သိချင်ရင်တော့ မော်ဒယ်က tag လုပ်ပေးထားတဲ့ဖိုင်ရဲ့ အတွင်းပိုင်းကို ဝင်ကြည့်တာပါပဲ။**
## Check otest.hyp File

In [66]:
!head -n 30 ../fst/otest.hyp

တစ်	tn
ကိုက်	v
ကို	ppm
ဝမ်	n
ခုနှစ်ထောင်	tn
ပါ	part
။	punc

မနှစ်	n
က	ppm
သူ	n
ကျွန်မ	pron
ကို	ppm
သင်	v
ပေး	v
တယ်	ppm
။	punc

ကျွန်တော့်	pron
ခုံ	n
သွား	v
ရှာ	v
မလို့	part
။	punc

အတန်း	n
စ	v
တာ	part
ကြာ	v
ပြီ	ppm


## Evaluation

WFST model က ဘယ်လောက်ထိ မှန်မှန်ကန်ကန် tagging လုပ်ပေးနိုင်သလဲ ဆိုတာကိုတော့ လူကိုယ်တိုင် tagging လုပ်ထားတဲ့ reference ဖိုင်နဲ့ တိုင်းတာကြည့်ရပါတယ်။  
အဲဒီလို တိုင်းတာနိုင်ဖို့အတွက် အရင်ဆုံး အထက်မှာ မြင်ရတဲ့အတိုင်း two column format ဖိုင်ကို ပုံမှန် line by line format ဖြစ်အောင် ပြောင်းလဲဖို့ လိုအပ်ပါတယ်။   
အဲဒီအတွက်ကိုတော့ col2line.pl (perl program) ကို သုံးပါမယ်။  
NLP preprocessing, post-editing အလုပ်အတွက် သုံးခဲ့တဲ့ script တချို့ကို GitHub မှာတင်ပေးထားပါတယ်။  
GitHub Link for Tools: [https://github.com/ye-kyaw-thu/tools](https://github.com/ye-kyaw-thu/tools)

In [67]:
!cat ../script/col2line.pl

#!/usr/bin/perl
use strict;
use utf8;

# last updated 16 May 2017
# written by Ye Kyaw Thu
# change column to line format

binmode(STDIN, ":utf8");
binmode(STDOUT, ":utf8");
binmode(STDERR, ":utf8");

open (col_FILE, "<:encoding(utf8)", $ARGV[0]) or die "Couldn\'t open input file $ARGV[0]!, $!\n";

my $tmpLine;
foreach my $line (<col_FILE>)
{
   chomp($line);

   if($line ne "")
   {
      $line =~ s/\t/\//;
      $tmpLine = $tmpLine.$line." ";
   }else
   {
      $tmpLine =~ s/\s+$//;
      print $tmpLine."\n";
      $tmpLine = "";
   }
}
close (col_FILE);


### Column to Line Format Conversion

In [68]:
!perl ../script/col2line.pl ../fst/otest.hyp > ../fst/otest.hyp.line

Perl program နဲ့ format ပြောင်းပြီး ထွက်လာတဲ့ ဖိုင်ကိုလည်း စစ်ဆေးကြည့်ရအောင်။  

In [69]:
!head ../fst/otest.hyp.line

တစ်/tn ကိုက်/v ကို/ppm ဝမ်/n ခုနှစ်ထောင်/tn ပါ/part ။/punc
မနှစ်/n က/ppm သူ/n ကျွန်မ/pron ကို/ppm သင်/v ပေး/v တယ်/ppm ။/punc
ကျွန်တော့်/pron ခုံ/n သွား/v ရှာ/v မလို့/part ။/punc
အတန်း/n စ/v တာ/part ကြာ/v ပြီ/ppm လား/part ။/punc
ဆေး/n နည်းနည်း/adj စား/v လိုက်/part ၊/punc သုံး/v လေး/part ရက်/n လောက်/part အနားယူ/v လိုက်/part ရင်/conj ပျောက်/v သွား/v မှာ/ppm ပါ/part ။/punc
အေးချမ်း/v မှု/part နဲ့/ppm စည်းကမ်း/n ကို/ppm တည်မြဲ/v အောင်/v ထိန်းသိမ်း/v သည်/ppm ။/punc
ဇွန်း/n ကို/ppm လိုအပ်/v တယ်/ppm ။/punc
ဘွဲ့/n ရ/v ရင်/conj ဘာ/adj လုပ်/v မ/part လို့/part လဲ/part ။/punc
ကျွန်တော်/pron ချောင်းဆိုး/v ခြင်း/part အတွက်/ppm တစ်/tn ခု/part ခု/part လို/v ချင်/part တယ်/ppm ။/punc
အသီးအနှံ/n တို့/part မှ/ppm လွဲ/v လျှင်/adj လူ/n တို့/part ၏/ppm အဓိက/n အစားအစာ/n မှာ/ppm ငါး/n ဖြစ်/v သည်/ppm ။/punc


**Reference ဖိုင်ကို သုံးပြီး Evaluation လုပ်ကြည့်မယ်။**  

In [73]:
!python ../script/pos_evaluation.py --ref ../data/otest.txt --hyp ../fst/otest.hyp.line

Accuracy: 91.66% (12345/13468)

Most common mistakes:
ရ/part --> ရ/v	91
ပြီး/conj --> ပြီး/v	73
ပေး/part --> ပေး/v	47
သွား/part --> သွား/v	36
ဒီ/pron --> ဒီ/adj	32
နှင့်/ppm --> နှင့်/conj	21
နဲ့/conj --> နဲ့/ppm	19
ဘာ/pron --> ဘာ/adj	18
နှစ်/tn --> နှစ်/n	18
လာ/v --> လာ/part	18


**--top-k ဆိုတဲ့ commandline option နဲ့ အမှားအများဆုံး အတွဲ ၃၀ ကို print ရိုက်ထုတ်ခိုင်းကြည့်ရအောင်။**  

In [74]:
!python ../script/pos_evaluation.py --ref ../data/otest.txt --hyp ../fst/otest.hyp.line --top-k 30

Accuracy: 91.66% (12345/13468)

Most common mistakes:
ရ/part --> ရ/v	91
ပြီး/conj --> ပြီး/v	73
ပေး/part --> ပေး/v	47
သွား/part --> သွား/v	36
ဒီ/pron --> ဒီ/adj	32
နှင့်/ppm --> နှင့်/conj	21
နဲ့/conj --> နဲ့/ppm	19
ဘာ/pron --> ဘာ/adj	18
နှစ်/tn --> နှစ်/n	18
လာ/v --> လာ/part	18
ဘယ်လောက်/adv --> ဘယ်လောက်/adj	16
ဘယ်/pron --> ဘယ်/adj	16
ဟာ/ppm --> ဟာ/n	16
ပြီး/part --> ပြီး/v	16
များ/adj --> များ/part	15
နဲ့/part --> နဲ့/ppm	14
သူ/pron --> သူ/n	13
ဟာ/part --> ဟာ/n	12
လျှင်/conj --> လျှင်/adj	11
နေ/v --> နေ/part	11
မှာ/v --> မှာ/ppm	11
ရန်/conj --> ရန်/n	11
ဘယ်လို/adv --> ဘယ်လို/adj	10
လေး/tn --> လေး/part	9
ရှိ/part --> ရှိ/v	9
သိပ်/adv --> သိပ်/v	9
ကြီး/adj --> ကြီး/part	8
ပါ/v --> ပါ/part	8
တောင်/part --> တောင်/n	8
၎င်း/pron --> ၎င်း/adj	8


## pos_evaluation.py
ဒီ python code က POS tagging အပြင်တခြား tagging evaluation တွေအတွက်လည်း အသုံးဝင်ပါလိမ့်မယ်။  

In [76]:
!cat ../script/pos_evaluation.py 

"""
Evaluation for POS Tagging, NER Tagging etc.
Written by Ye Kyaw Thu, LU Lab., Myanmar.
Last Update: 10 July 2025
Usage:
    python ./pos_evaluation.py --ref ../data/otest.txt --hyp ../fst/otest.hyp.line --top-k 30
"""

import sys
import argparse
from collections import defaultdict

def main():
    parser = argparse.ArgumentParser(description='Compare reference and hypothesis files for accuracy.')
    parser.add_argument('--ref', type=str, help='Reference file (default: stdin)')
    parser.add_argument('--hyp', type=str, help='Hypothesis file (default: stdin)')
    parser.add_argument('--top-k', type=int, default=10, 
                      help='Number of top mistakes to display (default: 10)')
    args = parser.parse_args()

    # Handle input files or stdin
    ref_file = sys.stdin if args.ref is None else open(args.ref, 'r', encoding='utf-8')
    hyp_file = sys.stdin if args.hyp is None else open(args.hyp, 'r', encoding='utf-8')

    mistakes = defaultdict(int)
    total = 0
    

## --help of pos_evaluation.py Code

In [78]:
!python ../script/pos_evaluation.py --help

usage: pos_evaluation.py [-h] [--ref REF] [--hyp HYP] [--top-k TOP_K]

Compare reference and hypothesis files for accuracy.

options:
  -h, --help     show this help message and exit
  --ref REF      Reference file (default: stdin)
  --hyp HYP      Hypothesis file (default: stdin)
  --top-k TOP_K  Number of top mistakes to display (default: 10)


## Building 3gram FST Model

အထက်က ရခဲ့တဲ့ ရလဒ်က 2gram Language Model နဲ့ပါ။  
ဒီတခါတော့ 3gram POS LM ဆောက်ကြည့်ပြီးတော့ ရလဒ်ဘယ်လောက်ထိ တက်လာသလဲ ဆိုတာကို လေ့လာကြည့်ရအောင်။  
အထက်ပါအတိုင်း command တွေ တစ်ခုချင်း ပေးပြီး run ရင်လည်း ရပေမယ့် ဒီတခါတော့ ရေးထားတဲ့ shell script ကိုသုံးပြီး အမြန် run သွားပါမယ်။  

In [79]:
!ls *.sh

build_fst.sh	    pipe2space.sh  rebuild.sh	  run_wfst_pos.sh
find_blank_line.sh  preprocess.sh  run_tagger.sh  test_tagger.sh


In [80]:
!cat ./rebuild.sh

#!/bin/bash

# Default to bigram, but allow override
NGRAM=${1:-2}

# Clean previous files
rm -f ../fst/*.fst ../fst/*.fst.txt

# Rebuild from scratch
./preprocess.sh
./build_fst.sh $NGRAM



**commandline argument အနေနဲ့ "3" ကို ရိုက်ထည့်ဖို့ မမေ့နဲ့ဦး။**  

In [81]:
!time ./rebuild.sh 3

mkdir: cannot create directory ‘../fst’: File exists
Preprocessing complete. Output files: ../fst/pairs.txt, ../fst/words.syms, ../fst/tags.syms
FST model built: pos_tagger.fst (using 3-gram POS LM)

real	0m1.195s
user	0m1.069s
sys	0m0.119s


In [82]:
!wc ../fst/pairs.txt ../fst/words.syms ../fst/tags.syms

  564517  1129034  9581272 ../fst/pairs.txt
   24827    49654   756965 ../fst/words.syms
      22       44      197 ../fst/tags.syms
  589366  1178732 10338434 total


In [83]:
!wc ../fst/*.fst

    1557     3942   441374 ../fst/lexicon.fst
    1557     3072   441374 ../fst/lexicon_sorted.fst
     287      523    34066 ../fst/pos_lm.fst
     287      538    34066 ../fst/pos_lm_sorted.fst
  236038  1089633 81434994 ../fst/pos_tagger.fst
  239726  1097708 82385874 total


In [84]:
!ls ../fst/*.txt

../fst/lexicon.fst.txt	../fst/pos_lm.fst.txt
../fst/pairs.txt	../fst/tagged_output.txt


In [88]:
!head -n 30 ../fst/pos_lm.fst.txt

0 1 num num 0
0 2 n n 0
0 3 v v 0
0 4 n n 0
0 5 ppm ppm 0
0 6 n n 0
0 7 num num 0
0 8 part part 0
0 9 v v 0
0 10 ppm ppm 0
0 11 punc punc 0
0 12 n n 0
0 13 part part 0
0 14 ppm ppm 0
0 15 v v 0
0 16 v v 0
0 17 part part 0
0 18 part part 0
0 19 n n 0
0 20 conj conj 0
0 21 conj conj 0
0 22 n n 0
0 23 punc punc 0
0 24 adj adj 0
0 25 ppm ppm 0
0 26 adv adv 0
0 27 part part 0
0 28 punc punc 0
0 29 pron pron 0
0 30 ppm ppm 0


## POS Tagging with 3-gram LM

In [89]:
!pwd

/home/ye/exp/wfst_pos/shell_script


In [90]:
!time ./run_tagger.sh ../data/otest.txt ../fst/words.syms ../fst/tags.syms ../fst/pos_tagger.fst > ../fst/otest_3gram.hyp


real	4m7.175s
user	3m34.467s
sys	0m45.863s


Tagging လုပ်တဲ့ process နှစ်ခုကို နှိုင်းယှဉ်ကြည့်ရင် 3-gram LM က 2-gram LM ထက် ပိုကြာတာကို တွေ့ရပါလိမ့်မယ်။ ၄မိနစ်နဲ့ ၇စက္ကန့်ကြာပါတယ်။  
ကောင်းပြီ။ 2-gram မော်ဒယ်ထက် tagging performance က ဘယ်လောက်ထိ တက်လာသလဲ ဆိုတာကို evaluation လုပ်ကြည့်ရအောင်။  

## Evaluation with 3-gram LM

အရင်ဆုံး blank file ဖြစ်မဖြစ် wc command နဲ့ စစ်ကြည့်ရအောင်။  

In [91]:
!wc ../fst/otest_3gram.hyp

 14468  26936 230687 ../fst/otest_3gram.hyp


File အတွင်းပိုင်းကိုလည်း မျက်စိနဲ့ တချက်ကြည့်ကြည့်ရအောင်...  

In [92]:
!head -n 30 ../fst/otest_3gram.hyp

တစ်	tn
ကိုက်	v
ကို	ppm
ဝမ်	n
ခုနှစ်ထောင်	tn
ပါ	part
။	punc

မနှစ်	n
က	ppm
သူ	n
ကျွန်မ	pron
ကို	ppm
သင်	v
ပေး	part
တယ်	ppm
။	punc

ကျွန်တော့်	pron
ခုံ	n
သွား	part
ရှာ	v
မလို့	part
။	punc

အတန်း	n
စ	v
တာ	part
ကြာ	v
ပြီ	ppm


**အခြေအနေကောင်းပုံ ရတယ်။  :)**  

column format ကနေ line format အဖြစ်ပြောင်းရဦးမယ်။  

In [98]:
!perl ../script/col2line.pl ../fst/otest_3gram.hyp > ../fst/otest_3gram.hyp.line

In [99]:
!head ../fst/otest_3gram.hyp.line

တစ်/tn ကိုက်/v ကို/ppm ဝမ်/n ခုနှစ်ထောင်/tn ပါ/part ။/punc
မနှစ်/n က/ppm သူ/n ကျွန်မ/pron ကို/ppm သင်/v ပေး/part တယ်/ppm ။/punc
ကျွန်တော့်/pron ခုံ/n သွား/part ရှာ/v မလို့/part ။/punc
အတန်း/n စ/v တာ/part ကြာ/v ပြီ/ppm လား/part ။/punc
ဆေး/n နည်းနည်း/adj စား/v လိုက်/part ၊/punc သုံး/v လေး/part ရက်/n လောက်/part အနားယူ/v လိုက်/part ရင်/conj ပျောက်/v သွား/part မှာ/ppm ပါ/part ။/punc
အေးချမ်း/v မှု/part နဲ့/ppm စည်းကမ်း/n ကို/ppm တည်မြဲ/v အောင်/conj ထိန်းသိမ်း/v သည်/ppm ။/punc
ဇွန်း/n ကို/ppm လိုအပ်/v တယ်/ppm ။/punc
ဘွဲ့/n ရ/v ရင်/part ဘာ/pron လုပ်/v မ/part လို့/part လဲ/part ။/punc
ကျွန်တော်/pron ချောင်းဆိုး/v ခြင်း/part အတွက်/ppm တစ်/tn ခု/part ခု/part လို/v ချင်/part တယ်/ppm ။/punc
အသီးအနှံ/n တို့/part မှ/ppm လွဲ/v လျှင်/conj လူ/n တို့/part ၏/ppm အဓိက/n အစားအစာ/n မှာ/ppm ငါး/n ဖြစ်/v သည်/ppm ။/punc


**Evaluation လုပ်မယ်။**  

In [101]:
!python ../script/pos_evaluation.py --ref ../data/otest.txt --hyp ../fst/otest_3gram.hyp.line

Accuracy: 94.00% (12660/13468)

Most common mistakes:
ရ/v --> ရ/part	35
ဒီ/pron --> ဒီ/adj	23
နဲ့/conj --> နဲ့/ppm	17
ပြီး/conj --> ပြီး/part	16
များ/adj --> များ/part	15
နှင့်/ppm --> နှင့်/conj	14
သွား/part --> သွား/v	13
ကောင်း/adj --> ကောင်း/v	12
ဘာ/adj --> ဘာ/pron	12
ပြီး/part --> ပြီး/conj	12


**ရလဒ်က တက်ပါတယ်။ ပထမ 2-gram LM နဲ့ run ထားတာကို Accuracy score ပြန်တွက်ကြည့်ရအောင်။**

In [102]:
!python ../script/pos_evaluation.py --ref ../data/otest.txt --hyp ../fst/otest.hyp.line

Accuracy: 91.66% (12345/13468)

Most common mistakes:
ရ/part --> ရ/v	91
ပြီး/conj --> ပြီး/v	73
ပေး/part --> ပေး/v	47
သွား/part --> သွား/v	36
ဒီ/pron --> ဒီ/adj	32
နှင့်/ppm --> နှင့်/conj	21
နဲ့/conj --> နဲ့/ppm	19
ဘာ/pron --> ဘာ/adj	18
နှစ်/tn --> နှစ်/n	18
လာ/v --> လာ/part	18


## Summary

WFST-based POS tagging ကို လက်တွေ့ လုပ်ပြခဲ့တယ်။  
ဒေတာက myPOS (version 3.0) ကို သုံးခဲ့တယ်။  
2-gram language model ကနေ 3-gram LM ကိုပြောင်းလိုက်တာနဲ့ POS tagging ရလဒ်က 91.66% ကနေ 94.00 ထိ တက်လာတာကို သက်သေပြခဲ့တယ်။  

Weighted Finite State Transducers ရဲ့ ကောင်းတဲ့အချက်က text file အဖြစ် မော်ဒယ်ကို print ရိုက်ထုတ်လိုက်ပြီး အသေးစိတ် debug လုပ်လို့ ရတာကြောင့် transparent ဖြစ်တယ်။  
အထူးသဖြင့် ဒေတာကပမာဏက နည်းရင် light-weight model တွေအနေနဲ့ဆောက်နိုင်တာမို့လို့ application အမျိုးမျိုးအတွက်လည်း အသုံးဝင်တယ်။
တော်တော်များများ NLP task တွေအတွက် top result မဟုတ်ရင်တောင်မှ လွယ်လွယ်ကူကူနဲ့ Accuracy က 90% ဝန်းကျင် ရရှိနိုင်တယ်။  
GPU မရှိလည်း အိုကေတယ်။  

တခုရှိတာက WFST တွေက အခြေခံအားဖြင့်က supervised learning ပါ။ ဆိုလိုတာက tagged လုပ်ထားတဲ့ ဒေတာ၊ reference ဒေတာ ရှိမှ အိုကေတယ်။  
တချို့ လက်တွေ့ အလုပ်တွေအတွက်က တခြား မော်ဒယ်တွေနဲ့ တွဲသုံးတာမျိုးလည်း လုပ်မှ ရလဒ်က ကောင်းတယ်။   

POS tagging လုပ်တာ အစအဆုံးကို ဆရာက လုပ်ပြထားလို့ လွယ်သရောင်ရောင် ထင်ရပေမဲ့ ကိုယ်တိုင် NLP task အလုပ်အသစ် တမျိုးမျိုးကို WFST နဲ့ လုပ်ကြည့်ရင် ဘယ်လောက် ခက်ခဲတယ် ဆိုတာကို သိလာပါလိမ့်မယ်။ အလုပ်တခု အစအဆုံး ပြီးစီးဖို့အတွက်က coding တွေ အများကြီးလုပ်ရပါတယ်။ ပြီးတော့ OpenFST command တွေက error message မပေးပဲ ရပ်သွားတာမို့ debug လုပ်ရတာ ခက်ခဲတာတွေလည်းရှိပါတယ်။ အဲဒါကြောင့် နောက်ဆုံးအနေနဲ့ မှာချင်တာက တကယ်တမ်း WFST နဲ့အလုပ်လုပ်နိုင်ဖို့ ဆိုရင် မတူတဲ့ ဒေတာကို သုံးပြီး မတူတဲ့ NLP task တခုခုကို လုပ်ကြည့်ကြပါလို့ အကြံပေးချင်ပါတယ်။    

